## Chat Models - Simple Chains


In [ ]:
import os
%pip install langchain
os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import PromptTemplate

In [ ]:
HumanMessagePromptTemplate.from_template("Test {product}")

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='Test {product}', template_format='f-string', validate_template=True), additional_kwargs={})

In [ ]:
# This is the long hand way of doing the same thing as the one below:
# human_message_prompt = HumanMessagePromptTemplate(
#     prompt=PromptTemplate(
#         template="What is a good name for a company that makes {product}?",
#         input_variables=["product"],
#     )
# )

human_message_prompt = HumanMessagePromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
# Creating the chat model:
chat = ChatOpenAI(temperature=0.9)

In [ ]:
# Creating the chain:
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))

Socktastic


---

## Multiple Input LLM Chains


In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(
    """Act as a social media manager for a company that sells {product}.
    You are creating a post for the company's social media account.

    Here is some additional context about the company: {company_context}

    You must follow the following principles:
    - The post must be 2 - 4 sentences in length.
    - The post must be written in a positive tone.
    - It must contain between 2 - 4 hashtags.

    Post:
    """
)


chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt])

# Creating the chain:
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

# Executing the chain:
chain(
    {
        "product": "colorful socks",
        "company_context": "The company sells socks that are colorful, and they are very popular with young people.",
    }
)

{'product': 'colorful socks',
 'company_context': 'The company sells socks that are colorful, and they are very popular with young people.',
 'text': '"Add a pop of color to your outfit with our vibrant and fun socks! 🌈🧦 Stand out from the crowd and express your unique style. Get ready to turn heads and put a smile on your face every time you look down at your feet! #ColorfulSocks #ExpressYourStyle #BeBold"'}

## Changing The Output Key:


In [ ]:
# Creating the chain:
chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key="post_text")

# Executing the chain:
chain(
    {
        "product": "colorful socks",
        "company_context": "The company sells socks that are colorful, and they are very popular with young people.",
    }
)

{'product': 'colorful socks',
 'company_context': 'The company sells socks that are colorful, and they are very popular with young people.',
 'post_text': "Get ready to step up your sock game! Our colorful socks are the perfect addition to any outfit, adding a pop of fun and style. With a wide range of vibrant patterns and comfortable materials, you'll be sure to turn heads wherever you go. #ColorfulSocks #SockGameStrong #FashionForward"}

---

## Async Methods


In [ ]:
import asyncio
import time
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def generate_serially():
    llm = ChatOpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="toothpaste")
        print(resp)


async def async_generate(chain):
    resp = await chain.arun(product="toothpaste")
    print(resp)


async def generate_concurrently():
    llm = ChatOpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


s = time.perf_counter()
# If running this outside of Jupyter, use asyncio.run(generate_concurrently())
await generate_concurrently()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")

DentaFresh
SmileFresh
PearlyWhite Dentifrice
GleamDent
1. Dentafresh
2. PureSmile
3. SmileSpark
4. BrightDent
5. PearlGlow
6. MintyFresh
7. PureWhite
8. OptiClean
9. FreshDent
10. PearlShine
Concurrent executed in 2.91 seconds.
FreshSmile
1. Pearly Whites Co.
2. FreshSmile Dental Care
3. MintyFresh Oral Care
4. SparklingDent Inc.
5. HappyBrush Toothpaste
6. SmileBright Dental Products
7. PureMint Toothpaste Co.
8. CleanDent Essentials
9. WhiteGlow Oral Care
10. FreshEssentials Dental Co.
PearlyFresh
PearlySmile
Some potential names for a toothpaste company could be:

1. PureSmile
2. SparkleDent
3. FreshGlow
4. MintVitality
5. PearlBright
6. OralEssentials
7. DentaFresh
8. SmileSavers
9. PearlMint
10. MintRevive
Serial executed in 7.86 seconds.


---

## Debugging chains with `verbose=True`


In [ ]:
# Creating the chain:
chain = LLMChain(
    llm=chat, prompt=chat_prompt_template, output_key="post_text", verbose=True
)

# Executing the chain:
chain(
    {
        "product": "colorful socks",
        "company_context": "The company sells socks that are colorful, and they are very popular with young people.",
    }
)



> Entering new LLMChain chain...
Prompt after formatting:
Human: What is a good name for a company that makes colorful socks?

> Finished chain.


{'product': 'colorful socks',
 'company_context': 'The company sells socks that are colorful, and they are very popular with young people.',
 'post_text': 'Vibrant Sock Co.\nRainbow Threads\nColorful Steps\nChromaSock\nSock Spectrum\nColorPop Socks\nColorBurst Sock Co.\nSockArt\nKaleidosock\nSpectraSock'}

You can also debug with by importing the langchain module and changing it to debug mode:


In [ ]:
import langchain

langchain.verbose = True

In [ ]:
# Creating the chain:
chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key="post_text")

# Executing the chain:
chain(
    {
        "product": "colorful socks",
        "company_context": "The company sells socks that are colorful, and they are very popular with young people.",
    }
)



> Entering new LLMChain chain...
Prompt after formatting:
System: Act as a social media manager for a company that sells colorful socks.
    You are creating a post for the company's social media account.

    Here is some additional context about the company: The company sells socks that are colorful, and they are very popular with young people.

    You must follow the following principles:
    - The post must be 2 - 4 sentences in length.
    - The post must be written in a positive tone.
    - It must contain between 2 - 4 hashtags.

    Post: 
    

> Finished chain.


{'product': 'colorful socks',
 'company_context': 'The company sells socks that are colorful, and they are very popular with young people.',
 'post_text': 'Step up your sock game with our vibrant and trendy collection of colorful socks! 🧦🌈 Perfect for adding a pop of color to any outfit and expressing your unique style. Get ready to turn heads and make a statement! #ColorfulSocks #SockGameStrong #ExpressYourStyle'}

In [ ]:
# Creating the chain:
chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key="post_text")

---


## Additional Ways of Running A Chain:


In [ ]:
input_list = [
    {
        "product": "colorful socks",
        "company_context": "The company sells socks that are colorful, and they are very popular with young people.",
    },
    {
        "product": "rock climbing shoes",
        "company_context": "The company sells rock climbing shoes that are colorful, and they are very popular with young people.",
    },
    {
        "product": "running shoes",
        "company_context": "The company sells running shoes that are colorful, and they are very popular with young people.",
    },
]

results = chain.apply(input_list)



> Entering new LLMChain chain...
Prompt after formatting:
Human: What is a good name for a company that makes colorful socks?
Prompt after formatting:
Human: What is a good name for a company that makes rock climbing shoes?
Prompt after formatting:
Human: What is a good name for a company that makes running shoes?

> Finished chain.


In [ ]:
results

[{'post_text': 'Spectrum Socks'},
 {'post_text': 'PeakProwler'},
 {'post_text': 'StrideElite'}]

In [ ]:
# If you use .generate(), then you will get an LLMResult object back.
results = chain.generate(input_list)

In [ ]:
results.llm_output

{'token_usage': {'prompt_tokens': 61,
  'completion_tokens': 10,
  'total_tokens': 71},
 'model_name': 'gpt-3.5-turbo'}

There are the .aapply() and .agenerate() methods for async.


---
